# Limpieza de carpetas de investigación del 2023

Estos datos corresponden al periodo del 01/Enero/2023 al 31/Marzo/2023

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sb
from sklearn.impute import SimpleImputer
import math
import re

## Entendimiento de los datos

In [2]:
df = pd.read_csv('data/da_2023.csv')
df.shape

(64344, 22)

In [3]:
df.columns

Index(['idCarpeta', 'Año_inicio', 'Mes_inicio', 'FechaInicio', 'Delito',
       'Categoria', 'Sexo', 'Edad', 'TipoPersona', 'CalidadJuridica',
       'competencia', 'Año_hecho', 'Mes_hecho', 'FechaHecho', 'HoraHecho',
       'HoraInicio', 'alcaldia_hechos', 'municipio_hechos', 'colonia_datos',
       'fgj_colonia_registro', 'latitud', 'longitud'],
      dtype='object')

In [4]:
#df[np.logical_and(df["colonia"].isnull(), df["persona"] == "MORAL")]

In [5]:
df.dtypes

idCarpeta                 int64
Año_inicio                int64
Mes_inicio               object
FechaInicio              object
Delito                   object
Categoria                object
Sexo                     object
Edad                    float64
TipoPersona              object
CalidadJuridica          object
competencia              object
Año_hecho               float64
Mes_hecho                object
FechaHecho               object
HoraHecho                object
HoraInicio               object
alcaldia_hechos          object
municipio_hechos         object
colonia_datos            object
fgj_colonia_registro     object
latitud                 float64
longitud                float64
dtype: object

In [6]:
df.nunique()

idCarpeta               61333
Año_inicio                  1
Mes_inicio                  3
FechaInicio                90
Delito                    255
Categoria                  16
Sexo                        2
Edad                      100
TipoPersona                 2
CalidadJuridica             5
competencia                 3
Año_hecho                  39
Mes_hecho                  12
FechaHecho               1434
HoraHecho                1441
HoraInicio               1455
alcaldia_hechos            18
municipio_hechos           91
colonia_datos            1655
fgj_colonia_registro     1538
latitud                 57654
longitud                57653
dtype: int64

In [7]:
df["Año_hecho"].describe()

count    64337.000000
mean      2022.731616
std          1.271916
min       1962.000000
25%       2023.000000
50%       2023.000000
75%       2023.000000
max       2023.000000
Name: Año_hecho, dtype: float64

### Tenemos delitos cometidos desde el año **1962** a los cuáles se les abrió carpetas de investigación hasta el año **2023**

In [8]:
df["Año_hecho"].unique()

array([2022., 2023., 2021., 2018., 2019., 2020., 2013., 1998., 2007.,
       2017., 2014., 2015., 2010., 2011.,   nan, 2003., 2016., 2005.,
       2000., 2006., 1993., 1995., 2012., 2002., 2008., 1969., 1987.,
       2009., 1967., 1990., 1999., 2004., 1994., 1974., 1976., 1985.,
       1962., 1991., 1972., 2001.])

In [9]:
df["Categoria"].unique()

array(['DELITO DE BAJO IMPACTO',
       'ROBO A TRANSEUNTE EN VÍA PÚBLICA CON Y SIN VIOLENCIA',
       'ROBO DE VEHÍCULO CON Y SIN VIOLENCIA', 'HECHO NO DELICTIVO',
       'LESIONES DOLOSAS POR DISPARO DE ARMA DE FUEGO', 'VIOLACIÓN',
       'ROBO A NEGOCIO CON VIOLENCIA',
       'ROBO A REPARTIDOR CON Y SIN VIOLENCIA', 'HOMICIDIO DOLOSO',
       'ROBO A PASAJERO A BORDO DE MICROBUS CON Y SIN VIOLENCIA',
       'ROBO A PASAJERO A BORDO DEL METRO CON Y SIN VIOLENCIA',
       'ROBO A PASAJERO A BORDO DE TAXI CON VIOLENCIA',
       'ROBO A CUENTAHABIENTE SALIENDO DEL CAJERO CON VIOLENCIA',
       'ROBO A CASA HABITACIÓN CON VIOLENCIA',
       'ROBO A TRANSPORTISTA CON Y SIN VIOLENCIA', 'SECUESTRO'],
      dtype=object)

In [10]:
df.loc[df["Año_hecho"]==1962]

,idCarpeta,Año_inicio,Mes_inicio,FechaInicio,Delito,Categoria,Sexo,Edad,TipoPersona,CalidadJuridica,...,Mes_hecho,FechaHecho,HoraHecho,HoraInicio,alcaldia_hechos,municipio_hechos,colonia_datos,fgj_colonia_registro,latitud,longitud
44757,9408996,2023,Marzo,2023-03-06,AMENAZAS,DELITO DE BAJO IMPACTO,Masculino,60.0,FISICA,VICTIMA Y DENUNCIANTE,...,Septiembre,1962-09-11,13:45:00,18:39:00,COYOACAN,NaN,DEL CARMEN,DEL CARMEN,19.351477,-99.158951


In [11]:
df["competencia"].unique()

array(['FUERO COMUN', 'HECHOS NO DELICTIVOS', 'INCOMPETENCIAS'],
      dtype=object)

### Resumen del entendimiento de los datos

1. Existen carpetas de investigación sobre delitos cometidos desde 1962, abiertas hasta el 2023.

2. Tenemos **16 categorías de delitos**.

3. Hay **5 tipos de calidades jurídicas**.

4. Describiendo los **3 competencias de delitos**:
    
    * Fuero común: Aquellos delitos que afectas directamente a la persona.
    * Hechos no delictivos: No son delitos.
    * Incompetencias: La persona es incapaz de ser juzgada o de ser responsable de sus acciones debido a condiciones físicas o mentales.
    
    
    
5. En la columna de alcaldías estamos considerando un **extra llamado 'fuera de la ciudad de méxico'**

6. Tenemos **255 tipos de delitos**.

## Limpieza

1. [X] Eliminar columnas que se relacionen con el tiempo en que se hizo la denuncia
2. [X] Estandarizar a minusculas las columnas restantes
3. [X] Renombrar columnas a 1 sola palabra
4. [X] Eliminar la palabra hecho de todas las columnas que la contengan
5. [X] Convertir a formato de fecha si corresponde
6. [X] Eliminar la columna de Municipio ya que solo vamos a analizar dentro de CDMX
7. [ ] Rellenar los NaN  

        7.1 [X] 'sexo' no, hasta tener separados los dataframes  
        
        7.2 [ ] 'edad' no, hasta tener separados los dataframes  
        
        7.3 [X] 'persona' no, eliminar los registros que no tengan este campo  
        
        7.4 [X] 'año, mes, fecha y hora' no, son 7 entonces eliminemos  
        
        7.5 [ ] 'colonia' si, quizas teniendo en cuenta las colonias que mas se repiten por alcaldia  
        
        7.6 [X] 'latitud y longitud' borrar columnas (realmente no se usarán)
        
8. [X] Cambiar a enteros los años
9. [X] Si existe un tipo de dato de "hora", cambiarlo en la que corresponda  

        9.1 [X] Eliminar aquellos registros con categoria "hecho no delictivo" y calidad "incompetencias"
        
10. [X] Separar en 2 DataFrames teniendo en cuenta el tipo de persona
11. [X] Del dataframe de personas morales, eliminar las columnas de sexo, edad, tipopersona
12. [ ] En la columna de edades de personas físicas, llenar los NA con algún **algoritmo de imputación**


... Si hay más, apúntelos aquí

### Paso 1

In [12]:
df.drop(
        columns=["fgj_colonia_registro", "idCarpeta", "Año_inicio", "Mes_inicio", "FechaInicio", "HoraInicio"], 
        inplace=True
        )

### Paso 2

In [13]:
df.rename(columns= lambda x: x.lower(), inplace=True)

### Paso 3

In [14]:
df.rename(columns = 
         {
             'tipopersona': 'persona',
             'calidadjuridica': 'calidad',
             'colonia_datos': 'colonia'
         }, 
        inplace=True)

### Paso 4

In [15]:
def find(name):
    return re.sub(r'_?hechos?$', '', name)

df.columns = df.columns.to_frame()[0].agg(find)

### Paso 5

In [16]:
df.fecha = pd.to_datetime(df.fecha)

### Paso 6: Eliminar Municipio

In [17]:
df.drop(columns="municipio", inplace=True)

### Paso 7.3, 7.4

In [18]:
# Borrar persona, año, mes fecha y hora nulos
df.dropna(subset=["persona", "año", "mes", "fecha", "hora"], inplace=True)

### Paso 7.6

In [19]:
df.drop(columns=["latitud", "longitud"], inplace=True)

### Paso 8

In [20]:
df["año"] = df['año'].astype(int)

### Paso 9

In [21]:
df["hora"] = pd.to_datetime(df['hora']).dt.time

### Paso 9.1

In [22]:
df = df[np.logical_and(df["categoria"] != "HECHO NO DELICTIVO", df["calidad"] != "INCOMPETENCIAS")]

### Paso 10: separacion de dataframe

In [23]:
df_fisica = df.query('persona == "FISICA"')
df_moral = df.query('persona == "MORAL"')

### Paso 11

In [24]:
df_moral = df_moral.drop(columns = ["edad", "sexo", "persona"])
df_moral.head(1)

,delito,categoria,calidad,competencia,año,mes,fecha,hora,alcaldia,colonia
0,LESIONES CULPOSAS POR TRANSITO VEHICULAR EN CO...,DELITO DE BAJO IMPACTO,OFENDIDO,FUERO COMUN,2022,Diciembre,2022-12-31,22:08:00,COYOACAN,INSURGENTES CUICUILCO


### Paso 12

In [25]:
df_fisica.isna().sum()

0
delito            0
categoria         0
sexo              0
edad           9737
persona           0
calidad           0
competencia       0
año               0
mes               0
fecha             0
hora              0
alcaldia          0
colonia        3184
dtype: int64

In [26]:
df_moral.isna().sum()

0
delito           0
categoria        0
calidad          0
competencia      0
año              0
mes              0
fecha            0
hora             0
alcaldia         0
colonia        709
dtype: int64

### Rellenado de colonias en moral

Agrupar por alcaldia y colonia

Seleccionar las mas ocurrentes en cada alcaldia y rellenarlo

In [27]:
# BORRAR REGISTROS QUE EN ALCALDIA CONTENGA CDMX Y FUERA DE CDMX
df_moral = df_moral[np.logical_and(df_moral["alcaldia"] != "CDMX", df_moral["alcaldia"] != "FUERA DE CDMX")]

In [28]:
imputer = SimpleImputer(strategy='most_frequent')

def imputacion(x):
    #print(x.values.reshape(-1, 1))
    return imputer.fit_transform(x.values.reshape(-1, 1)).ravel()

In [29]:
df_moral['colonia'] = df_moral.groupby('alcaldia')['colonia'].transform(imputacion)

In [30]:
df_moral.isna().sum()

0
delito         0
categoria      0
calidad        0
competencia    0
año            0
mes            0
fecha          0
hora           0
alcaldia       0
colonia        0
dtype: int64

### Imputación de las personas físicas

In [31]:
df_fisica.isna().count()

0
delito         53902
categoria      53902
sexo           53902
edad           53902
persona        53902
calidad        53902
competencia    53902
año            53902
mes            53902
fecha          53902
hora           53902
alcaldia       53902
colonia        53902
dtype: int64

In [32]:
df_fisica.groupby("alcaldia")["colonia"].count()

alcaldia
ALVARO OBREGON            3499
AZCAPOTZALCO              2383
BENITO JUAREZ             3705
CDMX                         0
COYOACAN                  3857
CUAJIMALPA DE MORELOS      940
CUAUHTEMOC                6655
FUERA DE CDMX                0
GUSTAVO A. MADERO         5173
IZTACALCO                 2597
IZTAPALAPA                7389
LA MAGDALENA CONTRERAS    1131
MIGUEL HIDALGO            3048
MILPA ALTA                 519
TLAHUAC                   1793
TLALPAN                   3091
VENUSTIANO CARRANZA       3071
XOCHIMILCO                1867
Name: colonia, dtype: int64

In [33]:
df_fisica = df_fisica[np.logical_and(df_fisica["alcaldia"] != "CDMX", df_fisica["alcaldia"] != "FUERA DE CDMX")]

In [34]:
df_fisica['colonia'] = df_fisica.groupby('alcaldia')['colonia'].transform(imputacion)

In [35]:
df_fisica.isna().sum()

0
delito            0
categoria         0
sexo              0
edad           8766
persona           0
calidad           0
competencia       0
año               0
mes               0
fecha             0
hora              0
alcaldia          0
colonia           0
dtype: int64

### Edades

In [36]:
medias_edades = df_fisica.groupby(by=["delito"])["edad"].mean().reset_index()
medias_edades.columns = ["delitos", "media_edad"]
#medias_edades[medias_edades["delitos"] == "ROBO DE OBJETOS"]

In [37]:
delitos_nulos = medias_edades[medias_edades["media_edad"].isna()]
delitos_completos = medias_edades[~medias_edades["media_edad"].isna()]

In [38]:
delitos_borrar = [x for x, row in delitos_nulos.iterrows()]

In [39]:
df_fisica = df_fisica[~df_fisica['delito'].isin(delitos_nulos)]

In [40]:
for index, row in df_fisica[["delito", "edad"]].iterrows():
    if not pd.notnull(row["edad"]):
        media_edad = medias_edades[medias_edades["delitos"] == row["delito"]]["media_edad"]
        df_fisica.at[index, "edad"] = media_edad


In [41]:
df_fisica.isna().sum()

0
delito          0
categoria       0
sexo            0
edad           87
persona         0
calidad         0
competencia     0
año             0
mes             0
fecha           0
hora            0
alcaldia        0
colonia         0
dtype: int64

In [42]:
df_fisica.dropna(inplace=True)

In [43]:
df_fisica["edad"] = df_fisica["edad"].astype(int)

In [44]:
df_fisica.drop(columns="persona", inplace = True)

In [45]:
df_fisica.isna().sum()

0
delito         0
categoria      0
sexo           0
edad           0
calidad        0
competencia    0
año            0
mes            0
fecha          0
hora           0
alcaldia       0
colonia        0
dtype: int64

### Exportando dataframes limpios

In [47]:
df_fisica.to_csv('data/fisicas.csv')

In [48]:
df_moral.to_csv('data/morales.csv')